In [ ]:
import json

# 打开并读取JSON文件
with open('dureadqg/train.json', 'r') as file:
    data = json.load(file)

# 输出读取到的数据
print(data)

In [1]:
import json

# 定义一个空列表用于存储所有的字典
train_list = []
test_list = []
# 打开JSON Lines文件
with open('dureadqg/train.json', 'r') as file:
    # 逐行读取
    for line in file:
        # 解析每一行
        data_dict = json.loads(line)
        # 将解析后的字典添加到列表中
        train_list.append(data_dict)
# 打开JSON Lines文件
with open('dureadqg/dev.json', 'r') as file:
    # 逐行读取
    for line in file:
        # 解析每一行
        data_dict = json.loads(line)
        # 将解析后的字典添加到列表中
        test_list.append(data_dict)


In [2]:
print(len(train_list), len(test_list))

14520 984


In [ ]:
!pip install datasets transformers rouge-score nltk

In [4]:
from datasets import *
train_dataset = Dataset.from_list(train_list)
test_dataset = Dataset.from_list(test_list)
print(train_dataset, test_dataset)

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['context', 'answer', 'question', 'id'],
    num_rows: 14520
}) Dataset({
    features: ['context', 'answer', 'question', 'id'],
    num_rows: 984
})


In [1]:
!git lfs install

Updated Git hooks.
Git LFS initialized.


In [2]:
!git clone https://huggingface.co/uer/t5-base-chinese-cluecorpussmall

正克隆到 't5-base-chinese-cluecorpussmall'...
fatal: 无法访问 'https://huggingface.co/uer/t5-base-chinese-cluecorpussmall/'：Failed to connect to huggingface.co port 443: 连接超时


In [3]:
from modelscope.hub.api import HubApi

YOUR_ACCESS_TOKEN = '49834fc2-f98f-4ac3-8efc-01ce1764baf8'

api = HubApi()
api.login(YOUR_ACCESS_TOKEN)
api.push_model(
    model_id="zlyzly/t5-base-chinese-cluecorpussmall", 
    model_dir="t5-base-chinese-cluecorpussmall" # 本地模型目录，要求目录中必须包含configuration.json
)

2024-07-24 10:21:54,294 - modelscope - INFO - [master ee39e78] 'upload model'
 5 files changed, 21385 insertions(+)
 create mode 100644 README.md
 create mode 100644 config.json
 create mode 100644 special_tokens_map.json
 create mode 100644 tokenizer_config.json
 create mode 100644 vocab.txt



In [5]:
from modelscope import HubApi
from modelscope import snapshot_download

api=HubApi()
api.login('49834fc2-f98f-4ac3-8efc-01ce1764baf8')

# download your model, the model_path is downloaded model path.
model_path =snapshot_download(model_id='zlyzly/t5-base-chinese-cluecorpussmall')


2024-07-24 10:40:20,076 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2024-07-24 10:40:20,078 - modelscope - INFO - TensorFlow version 2.13.0 Found.
2024-07-24 10:40:20,079 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-07-24 10:40:20,079 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-07-24 10:40:20,239 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 e096d041fdba41d8d7d7331b3f469288 and a total number of 945 components indexed
Downloading: 100%|██████████| 460/460 [00:00<00:00, 51.4kB/s]
Downloading: 100%|██████████| 26.0/26.0 [00:00<00:00, 3.95kB/s]
Downloading: 100%|█████████▉| 818M/818M [00:02<00:00, 286MB/s] 
Downloading: 100%|██████████| 6.32k/6.32k [00:00<00:00, 3.38MB/s]
Downloading: 100%|██████████| 263/263 [00:00<00:00, 142kB/s]
Downloading: 100%|██████████| 108k/108k [00:00<00:00, 19.7MB/s]


In [6]:
model_path

'/mnt/workspace/.cache/modelscope/zlyzly/t5-base-chinese-cluecorpussmall'

In [7]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
tokenizer = AutoTokenizer.from_pretrained(model_path)

2024-07-24 10:40:33.418967: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-24 10:40:33.838789: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 10:40:34.962499: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [9]:
model

T5ForConditionalGeneration(
  (shared): Embedding(21228, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(21228, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): FusedRMSNorm(torch.Size([768]), eps=1e-06, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_